<a href="https://colab.research.google.com/github/mikeornstein/lets-meet-halfway/blob/main/lets_meet_halfway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install descartes

In [8]:
import requests
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, mapping
from descartes import PolygonPatch
from google.colab import userdata

# Define your API key for Open Route Service
api_key = userdata.get('secretORS')

# Define function to make request to Open Route Service API and get isochrone
def get_isochrone(location, range_values):
    url = f"https://api.openrouteservice.org/v2/isochrones/driving-car"
    headers = {"Authorization": f"Bearer {api_key}"}
    params = {
        "locations": [location],
        "range": range_values,
        "interval": 600,
        "units": "m",
    }
    response = requests.post(url, json=params, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.text)
        return None

# Define function to plot isochrone
def plot_isochrone(location, isochrone):
    plt.figure(figsize=(8, 8))
    ax = plt.gca()

    for feature in isochrone['features']:
        polygon = Polygon(feature['geometry']['coordinates'][0])
        patch = PolygonPatch(polygon, edgecolor='blue', facecolor='blue', alpha=0.3)
        ax.add_patch(patch)

    ax.set_aspect('equal')
    plt.title(f'Isochrone around {location}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.grid(True)
    plt.show()

def get_lat_lon_from_zip(zip_code, api_key):
    url = f"https://api.openrouteservice.org/geocode/search?api_key={api_key}&text={zip_code}"
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data["features"]:
            lat = data["features"][0]["geometry"]["coordinates"][1]
            lon = data["features"][0]["geometry"]["coordinates"][0]
            return lat, lon
        else:
            print("No results found for the provided ZIP code.")
            return None
    else:
        print("Error:", response.text)
        return None

Latitude: 34.100517, Longitude: -118.41463


In [ ]:
# Example usage:
zip_code = "90210"  # Example ZIP code
latitude, longitude = get_lat_lon_from_zip(zip_code, api_key)
if latitude and longitude:
    print(f"Latitude: {latitude}, Longitude: {longitude}")

# Define your locations
location1 = [lon1, lat1]  # coordinates of location 1, e.g., [longitude, latitude]
location2 = [lon2, lat2]  # coordinates of location 2, e.g., [longitude, latitude]

# Define the range values for isochrones (in seconds)
range_values = [300, 600, 900]  # for example, 5, 10, and 15 minutes

# Get isochrones for both locations
isochrone_location1 = get_isochrone(location1, range_values)
isochrone_location2 = get_isochrone(location2, range_values)

# Plot isochrones
if isochrone_location1:
    plot_isochrone("Location 1", isochrone_location1)

if isochrone_location2:
    plot_isochrone("Location 2", isochrone_location2)
